In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# import tensorflow_recommenders as tfrs

In [4]:
# ratings = pd.read_pickle("tiny_selection_longform.pkl")
ratings = pd.read_pickle("./Data/moderate_users_longform.pkl")
user_stats = pd.read_pickle("./Data/moderate_user_stats.pkl")

In [5]:
user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['user_id'].values)
n_users = ratings['user'].nunique()
n_users

141676

In [6]:
item_enc = LabelEncoder()
ratings['movie'] = item_enc.fit_transform(ratings['movie_id'].values)
n_movies = ratings['movie'].nunique()
n_movies

72601

In [7]:
ratings.head()

,movie_id,rating_score,user_id,user,movie
5,1066,4,97262846,137735,1023
8,1066,4,58420503,83022,1023
11,1066,2,15492642,22070,1023
12,1066,4,29755671,42177,1023
15,1066,3,93302487,132199,1023


In [8]:
def center_ratings(ratings_df, users_df):
    ratings_df['centered_rating'] = None
    for row in ratings_df.index:
        mean_val = users_df.at[ratings_df.at[row, 'user_id'], 'mean_score']
        ratings_df.at[row, 'centered_rating'] = ratings_df.at[row, 'rating_score'] - mean_val
        
    return ratings_df
        

In [9]:
ratings_centered = center_ratings(ratings, user_stats)

In [10]:
ratings_centered.head()

,movie_id,rating_score,user_id,user,movie,centered_rating
5,1066,4,97262846,137735,1023,0.324786
8,1066,4,58420503,83022,1023,0.0
11,1066,2,15492642,22070,1023,-1.84
12,1066,4,29755671,42177,1023,-0.857143
15,1066,3,93302487,132199,1023,-0.666667


In [11]:

X = ratings_centered[['movie', 'user']].values
y = ratings_centered['centered_rating'].values

X.shape, y.shape

((6864260, 2), (6864260,))

In [12]:
#model using functional API -- not really working 

inputs = keras.Input(shape=(2,))

dense = layers.Dense(10, activation='relu')
x = dense(inputs)

x = layers.Dense(2, activation='relu')(x)
outputs = keras.activations.softmax(x, axis=1)

In [13]:
model = keras.Model(inputs=inputs, outputs=outputs, name='testing')

In [14]:
model.summary()

Model: "testing"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
_________________________________________________________________
tf.compat.v1.nn.softmax (TFO (None, 2)                 0         
Total params: 52
Trainable params: 52
Non-trainable params: 0
_________________________________________________________________


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y.astype('float32'), test_size=0.33, random_state=42)

In [16]:
model.compile(optimizer = keras.optimizers.Adam(learning_rate=.1),
             loss = keras.losses.MeanSquaredError(),
            metrics = keras.metrics.RootMeanSquaredError())

In [17]:
model.fit(X_train, y_train, batch_size=1000, epochs=3, verbose=1, validation_split=.2)

Epoch 1/3
3680/3680 [==============================] - 12s 3ms/step - loss: 1.0639 - root_mean_squared_error: 1.0315 - val_loss: 1.0620 - val_root_mean_squared_error: 1.0305
Epoch 2/3
3680/3680 [==============================] - 10s 3ms/step - loss: 1.0639 - root_mean_squared_error: 1.0315 - val_loss: 1.0620 - val_root_mean_squared_error: 1.0305
Epoch 3/3
3680/3680 [==============================] - 10s 3ms/step - loss: 1.0639 - root_mean_squared_error: 1.0315 - val_loss: 1.0620 - val_root_mean_squared_error: 1.0305


In [18]:
test_scores = model.evaluate(X_test, y_test, verbose=1)

70788/70788 [==============================] - 99s 1ms/step - loss: 1.0640 - root_mean_squared_error: 1.0315


In [19]:
test_scores

[1.0639585256576538, 1.0314836502075195]

In [22]:
ratings[ratings.user_id==93302487]

,movie_id,rating_score,user_id,user,movie,centered_rating
15,1066,3,93302487,132199,1023,-0.666667
177284,1345,3,93302487,132199,1275,-0.666667
198923,1359,5,93302487,132199,1288,1.333333
244847,1399,4,93302487,132199,1325,0.333333
317119,1465,4,93302487,132199,1386,0.333333
...,...,...,...,...,...,...
15127107,865,3,93302487,132199,829,-0.666667
15343609,963,3,93302487,132199,924,-0.666667
15399840,978,4,93302487,132199,939,0.333333
15452593,995,5,93302487,132199,956,1.333333


In [28]:
ratings.at[15, 'movie']

1023

In [30]:
for row in ratings[ratings.user_id==93302487].index:
    movie = ratings.at[row, 'movie']
    pred = model.predict([[int(movie), 132199]])
    print(movie, ratings.at[row, 'centered_rating'], pred)

1023 -0.6666666666666665 [[0.5 0.5]]
1275 -0.6666666666666665 [[0.5 0.5]]
1288 1.3333333333333335 [[0.5 0.5]]
1325 0.3333333333333335 [[0.5 0.5]]
1386 0.3333333333333335 [[0.5 0.5]]
1427 -0.6666666666666665 [[0.5 0.5]]
1524 -1.6666666666666665 [[0.5 0.5]]
1586 -0.6666666666666665 [[0.5 0.5]]
1599 0.3333333333333335 [[0.5 0.5]]
1650 0.3333333333333335 [[0.5 0.5]]
1655 -1.6666666666666665 [[0.5 0.5]]
2136 0.3333333333333335 [[0.5 0.5]]
2389 0.3333333333333335 [[0.5 0.5]]
3100 -0.6666666666666665 [[0.5 0.5]]
3104 0.3333333333333335 [[0.5 0.5]]
3106 -0.6666666666666665 [[0.5 0.5]]
3123 0.3333333333333335 [[0.5 0.5]]
4911 -0.6666666666666665 [[0.5 0.5]]
6271 -0.6666666666666665 [[0.5 0.5]]
7858 -0.6666666666666665 [[0.5 0.5]]
3 0.3333333333333335 [[0.5 0.5]]
59 0.3333333333333335 [[0.5 0.5]]
67 0.3333333333333335 [[0.5 0.5]]
81 0.3333333333333335 [[0.5 0.5]]
83 0.3333333333333335 [[0.5 0.5]]
84 0.3333333333333335 [[0.5 0.5]]
99 1.3333333333333335 [[0.5 0.5]]
108 0.3333333333333335 [[0.5 0.5